# Image Windowing and Matrix Factorization

### Rama Vasudevan

### Px Credits: Gerd Duscher, Suhas Somnath, S. Mani Valleti, Maxim Ziatdinov, Rama Vasudevan

Let's look at some image analytics here

In [ ]:
%matplotlib notebook
!pip install -U pyNSID sidpy SciFiReaders nanonispy gwyfile pycroscopy
#!pip install pip install threadpoolctl==3.1.0

In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np

import pyNSID
import sidpy as sid
import SciFiReaders as sr
from sidpy.proc.fitter import SidFitter

# Second example - Image analysis

The pycroscopy package has some tools for generic image analysis, as well as wrappers around common machine learning methods. These include matrix and tensor factorization techniques. Let us explore one example.

First we will import a microscopy image, and then we will perform image windowing

We will then use matrix factorization to analyze the spatial distribution of different phases

This is explained in <a href = "https://pubs.acs.org/doi/full/10.1021/acs.nanolett.6b02130">this article</a>.


In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import numpy as np
import pyNSID

import sidpy as sid
import SciFiReaders as sr

In [ ]:
dm3_file = r'data/bto_atomic.dm3'
png_file = r'data/re48.tiff'

dm3_reader = sr.DM3Reader(dm3_file)
img_reader = sr.ImageReader(png_file)

data = dm3_reader.read()

In [ ]:
data.plot();

In [ ]:
#Let's crop the image and try again
# Make a sidpy dataset

x_dim = data.dim_0.values
y_dim = data.dim_1.values

data_cropped = data[:][200:-200,10:550]

data_set = sid.Dataset.from_array(data_cropped, title='BTO_STEM')

# Set the data type
data_set.data_type = sid.DataType.IMAGE

# Add quantity and units
data_set.units = 'counts'
data_set.quantity = 'Intensity'

y_dim = y_dim[10:550]
x_dim = x_dim[200:-200]

# Add dimension info
data_set.set_dimension(0, sid.Dimension(x_dim,
                                        name='x',
                                        units='nm', quantity='x',
                                        dimension_type='spatial'))
data_set.set_dimension(1, sid.Dimension(y_dim,
                                        name='y',
                                        units='nm', quantity='y',
                                        dimension_type='spatial'))


In [ ]:
data_set.plot();

In [ ]:
from pycroscopy.image import ImageWindowing

parms_dict = {}
parms_dict['window_step_x'] = 16
parms_dict['window_step_y'] = 16
parms_dict['window_size_x'] = 64
parms_dict['window_size_y'] = 64
parms_dict['mode'] = 'fft'
parms_dict['filter'] = 'hamming'
parms_dict['zoom_factor'] = 2
parms_dict['interpol_factor'] = 1
iw = ImageWindowing(parms_dict)
windows = iw.MakeWindows(data_set)
windows = np.abs(np.log(np.abs(windows)))



In [ ]:
%matplotlib notebook
windows.plot();

In [ ]:
from pycroscopy.learn.ml.matrix_factor import MatrixFactor
mfactor = MatrixFactor(np.abs(windows), method = 'pca',n_components = 3 )
output = mfactor.do_fit()

In [ ]:
abundances = output[0]
components = output[1]
abund = np.array(abundances)
comps = np.array(components)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, axes = plt.subplots(nrows=1, ncols=mfactor.ncomp, figsize = (10,3))
for ind, ax in enumerate(axes.flat):
    im1 = ax.imshow(comps[ind,:,:])
    ax.set_title('Component #' + str(ind))
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')
    ax.axis('off')
fig.tight_layout()

fig, axes = plt.subplots(nrows=1, ncols=mfactor.ncomp, figsize = (10,3))
for ind, ax in enumerate(axes.flat):
    im1 = ax.imshow(abund[:,:,ind])
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(im1, cax=cax, orientation='vertical')
    ax.axis('off')

fig.tight_layout()

## Exercises

1. Play with the number of components present and see how it affects the result. Similarly, play with the different types of factorization ('nmf', 'pca', 'nfindr'). How do the results differ?

2. Think about how we might want to determine how many components we should use. HINT: Look at reconstruction errors, or similarity of the components.

[RKV to do: write the code for structural similarity of the components, if any two have a similarity beyond a certain level then we can cut off there]